# ETL

Parts of this rationale are stored in the Architectural Decision Document (ADD) already.

My project is to predict fantasy football performances given past player and team performances.  My data source comes from the pro football reference website as well as the NFL website.
The season fantasy data is small and easy to save individually.
Individual player's data come in different sized tables.  Even for the same position, the fields are not necessarily the same.
	The columns are read from the html and assigned to each table.
A concatenation of all tables at the end of the scraping is then done.

Additional Data
	In the future, I'd like to add more stats
		- college stats
			- college team ranking
		- dome / field surface type
		- weather conditions during game times
		- coaching staff (categorical)

## Extract

Extracting team season information
	This dataset is small enough that each year is saved manually

Extracting player season information
	This dataset is small enough that each year is saved manually


In [61]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

## Transform

For each of the season data, a trailing 3y average is taken for both player and team data.
Unfortunately, we are working with Spark 2.1 which can't do this easily.

https://stackoverflow.com/questions/33207164/spark-window-functions-rangebetween-dates

I have created the rolling averages in pandas.

Individual player game data will be looked at separately

In [22]:
import pandas as pd
import numpy as np

In [5]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=49F92286A8610A74&resid=49F92286A8610A74%217441&authkey=ABEh6XBdaAPm4ls" -O players_stats_proFBRef.csv

--2018-12-25 11:29:42--  https://onedrive.live.com/download?cid=49F92286A8610A74&resid=49F92286A8610A74%217441&authkey=ABEh6XBdaAPm4ls
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lt0peg.bn.files.1drv.com/y4meXhN1NLFkAidr5fhBGMPLeYpQEMSfsf5EOMfos2Ev4ELbMJSOC5QF4H5QzHFirTtsY70CUHHhI_yr7NSC1X2yD6JA64T5QU6Bs8Koi5e1Oued3HmsAeTR5TN5rVX_hSKNhLZwaKFCSTDTSYcyX12cQpgZmJAxMF0cCTA7U5gQsYXg2-z5vncKpjQWL_gyR5N004NdNe1hAWxH3LFjyY4lA/all_player_stats_proFBref.csv?download&psid=1 [following]
--2018-12-25 11:29:43--  https://lt0peg.bn.files.1drv.com/y4meXhN1NLFkAidr5fhBGMPLeYpQEMSfsf5EOMfos2Ev4ELbMJSOC5QF4H5QzHFirTtsY70CUHHhI_yr7NSC1X2yD6JA64T5QU6Bs8Koi5e1Oued3HmsAeTR5TN5rVX_hSKNhLZwaKFCSTDTSYcyX12cQpgZmJAxMF0cCTA7U5gQsYXg2-z5vncKpjQWL_gyR5N004NdNe1hAWxH3LFjyY4lA/all_player_stats_proFBref.csv?download&psid=1
Resolving lt0peg.bn.files.

In [6]:
player_seasons = pd.read_csv('players_stats_proFBRef.csv')
nfl_seasons = pd.read_csv('NFL_team_stats.csv')

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3,5,6,12,13,14,15,16,18,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
player_seasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72167 entries, 0 to 72166
Data columns (total 75 columns):
Unnamed: 0               72167 non-null int64
Unnamed: 0.1             72167 non-null int64
Gen-Rk                   72164 non-null object
Gen-Year                 72164 non-null object
Gen-Date                 72167 non-null object
Gen-G#                   72164 non-null object
Gen-Age                  72164 non-null object
Gen-Tm                   72164 non-null object
Gen-Unnamed: 0           36038 non-null object
Gen-Opp                  72164 non-null object
Gen-Result               72167 non-null object
Games-GS                 42345 non-null object
Receiving-Tgt            63974 non-null object
Receiving-Rec            63877 non-null object
Receiving-Yds            63870 non-null object
Receiving-Y/R            46029 non-null object
Receiving-TD             63870 non-null object
Receiving-Ctch%          67457 non-null object
Receiving-Y/Tgt          49438 non-null object


In [17]:
player_seasons['Gen-Year'] = pd.to_numeric(player_seasons['Gen-Year'], downcast='integer', errors='coerce')

### Data Cleaning

In [29]:
# where there is no team, the player has been on more than 1 team during a season and the stats are split out
# for the purposes of this analysis, we will exclude those lines.
# there are also some Tm lines to get rid of
player_seasons = player_seasons[[not(pd.isnull(tm)) for tm in player_seasons['Gen-Tm']]]
player_seasons = player_seasons[player_seasons['Gen-Tm'] != 'Tm']

In [31]:
print(sorted(list(set(player_seasons['Gen-Tm']))))
print(sorted(list(set(nfl_seasons.Team_Short))))

['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GNB', 'HOU', 'IND', 'JAX', 'KAN', 'LAC', 'LAR', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OAK', 'PHI', 'PHO', 'PIT', 'RAM', 'SDG', 'SEA', 'SFO', 'STL', 'TAM', 'TEN', 'WAS']
['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']


In [33]:
# Match up team abbreviations between pro football reference (PFR) and NFL.  The NFL has changed the team name retrospectively if a team has moved.
# PFR keeps a convention of 3 letters per team and the NFL doesn't
team_abbrev_pfr_nfl = {'GNB': 'GB',                      
                       'KAN': 'KC',                      
                       'NOR': 'NO',                      
                       'NWE': 'NE',                      
                       'SDG': 'LAC',
                       'STL': 'LAR',
                       'SFO': 'SF',
                       'TAM': 'TB'}
player_seasons['Gen-Tm'] = [team_abbrev_pfr_nfl[x] if x in team_abbrev_pfr_nfl else x for x in player_seasons['Gen-Tm']]

In [40]:
# combine player and team data
'''
all_df = pd.merge(fant_seasons, nfl_seasons, how='left', left_on=['Year','Tm'], right_on=['Year','Team_Short'])
all_df.to_csv(r'pfr_nfl.csv')
'''
df = player_seasons

In [109]:
ff_pts_map = {
'Receiving-Rec': {'pts': 0.5, 'per':1},
'Passing-Yds': {'pts': 1, 'per':25},
'Passing-TD': {'pts': 4, 'per':1},
'Passing-Int': {'pts': -1, 'per':1},
'Rushing-Yds': {'pts': 1, 'per':10},
'Rushing-TD': {'pts': 6, 'per':1},
'Receiving-Yds': {'pts': 1, 'per':10},
'Receiving-TD': {'pts': 6, 'per':1},
'Receiving-TD': {'pts': 6, 'per':1},
'Kick Returns-TD': {'pts': 6, 'per':1},
'Punt Returns-TD': {'pts': 6, 'per':1},
'Fumbles-TD': {'pts': 6, 'per':1},
'Fumbles-Fmb': {'pts': -2, 'per':1}
}

In [114]:
import pdb
def calc_ff_pts(df_row):
    '''
    loops through each way to score points and add
    :param df_row: the row of the dataframe that you need to calc
    '''    
    points = 0    
    for f in ff_pts_map:
        #pdb.set_trace()
        if pd.isnull(df_row[f]):
            fld = 0
        else:
            fld = float(df_row[f])
        points += fld * ff_pts_map[f]['pts'] / ff_pts_map[f]['per']
        #print(f, fld, ff_pts_map[f]['pts'], ff_pts_map[f]['per'], points)
    return points

In [115]:
df['ff_pts'] = df.apply(lambda row: calc_ff_pts(row), axis=1)

In [116]:
df.sample(50)

,Unnamed: 0,Unnamed: 0.1,Gen-Rk,Gen-Year,Gen-Date,Gen-G#,Gen-Age,Gen-Tm,Gen-Unnamed: 0,Gen-Opp,Gen-Result,Games-GS,Receiving-Tgt,Receiving-Rec,Receiving-Yds,Receiving-Y/R,Receiving-TD,Receiving-Ctch%,Receiving-Y/Tgt,Rushing-Att,Rushing-Yds,Rushing-Y/A,Rushing-TD,Scoring-TD,Scoring-Pts,Def Interceptions-Int,Def Interceptions-Yds,Def Interceptions-TD,Def Interceptions-PD,Fumbles-Fmb,Fumbles-FF,Fumbles-FR,Fumbles-Yds,Fumbles-TD,Name,Posn,DOB,College,Passing-Cmp,Passing-Att,Passing-Cmp%,Passing-Yds,Passing-TD,Passing-Int,Passing-Rate,Passing-Sk,Passing-Yds.1,Passing-Y/A,Passing-AY/A,Scoring-2PM,Kick Returns-Rt,Kick Returns-Yds,Kick Returns-Y/Rt,Kick Returns-TD,Gen-Sk,Tackles-Solo,Tackles-Ast,Tackles-Comb,Tackles-TFL,Tackles-QBHits,Punt Returns-Ret,Punt Returns-Yds,Punt Returns-Y/R,Punt Returns-TD,Punting-Pnt,Punting-Yds,Punting-Y/P,Punting-Blck,Scoring-Sfty,Scoring-XPM,Scoring-XPA,Scoring-XP%,Scoring-FGM,Scoring-FGA,Scoring-FG%,ff_pts
21520,8,8,9,2016.0,2016-11-06,9,24.163,DEN,@,OAK,L 20-30,*,3,1,8,8,0,33.3%,2.67,10.0,22.0,2.20,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Devontae Booker,RB,1992-05-27,Utah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.50
35671,22,25,23,2017.0,2017-12-31,16,26.059,SF,@,LAR,W 34-13,*,0,0,0,NaN,0,0.0%,NaN,2.0,-1.0,-0.50,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Jimmy Garoppolo,QB,1991-11-02,East. Illinois,20.0,33.0,60.61,292.0,2.0,2.0,84.4,0.0,0.0,8.85,7.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.58
34571,89,95,90,2018.0,2018-10-14,6,28.25,NYJ,NaN,IND,W 42-34,*,10,9,94,10.44,0,90.0%,9.4,0.0,0.0,NaN,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Jermaine Kearse,WR,1990-02-06,Washington,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.90
64548,78,83,79,2009.0,2009-11-22,10,26.123,LAR,NaN,ARI,L 13-21,*,2,2,0,0,0,100.0%,0,24.0,116.0,4.83,1.0,1,6,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Steven Jackson,RB,1983-07-22,Oregon St.,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.60
16487,42,45,43,2014.0,2014-10-19,6,26.355,MIA,@,CHI,W 27-14,NaN,3,3,25,8.33,0,100.0%,8.33,7.0,25.0,3.57,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Daniel Thomas,RB,1987-10-29,Kansas St.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.50
25438,28,30,29,2012.0,2012-10-11,5,25.208,PIT,@,TEN,L 23-26,NaN,6,4,43,10.75,0,66.7%,7.17,0.0,0.0,NaN,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Emmanuel Sanders,WR,1987-03-17,SMU,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.30
25082,126,134,127,2007.0,2007-12-23,15,29.144,ARI,NaN,ATL,W 30-27,*,5,4,29,7.25,0,80.0%,5.8,26.0,68.0,2.62,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Edgerrin James,RB,1978-08-01,Miami (FL),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.70
41506,67,72,68,2000.0,2000-10-08,6,27.283,NYG,@,ATL,W 13-6,*,0,0,0,NaN,0,0.0%,NaN,4.0,5.0,1.25,0.0,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Kerry Collins,QB,1972-12-30,Penn St.,14.0,25.0,56.00,151.0,0.0,2.0,40.6,0.0,0.0,6.04,2.44,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.54
66569,110,117,111.0,2007.0,2007-11-22,11.0,29.095,NYJ,@,DAL,L 3-34,*,3.0,2.0,17.0,8.5,0.0,66.7%,5.67,17.0,40.0,2.35,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,Thomas Jones,RB,1978-08-19,Virginia,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [117]:
df.to_csv(r'all_players_w_features.csv')

## Load

In [118]:
from pyspark.sql import SQLContext
from pyspark.conf import SparkConf
SparkSession.builder.config(conf=SparkConf())

In [119]:
sdf = (spark.read.format("csv").options(header="true").load("all_players_w_features.csv"))
rdd = sdf.rdd